In [ ]:
# -*- coding:utf-8 -*-
# !/usr/bin/python
# Author: Selvaria
# 获取岛内企业的上下游开票情况并存储，接受excel和单独名称两种格式

import pymysql
import xlrd
import xlwt


connection = pymysql.connect(host='47.92.25.70', user='root', passwd='Wfn031641',
                db='cxd_data', port=3306, charset='utf8')
cursor = connection.cursor()

path = './files/company.xls'
data = xlrd.open_workbook(path) # 打开xls文件
table = data.sheets()[0]  # 打开第一张表
nrows = table.nrows      # 获取表的行数
com_info = []
for i in range(nrows):   # 循环逐行打印
    if i == 0:  # 跳过第一行
        continue
    com_name = table.row_values(i)[0].strip() #第一列是公司名字是填零，之后类推
    if com_name not in com_info:
        com_info.append(com_name)

sql_up = "select Xf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
         "financial_exchange where Gf_company_name = '%s' and year = '2018'"
sql_down = "select Gf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
         "financial_exchange where Xf_company_name = '%s' and year = '2018'" 
    
#sql_up = "select Xf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \  #特殊要求时exchange_good = '柴油 -35#'
#         "financial_exchange where Gf_company_name = '%s' and exchange_good = '柴油 -35#' and year = '2018'"
#sql_down = "select Gf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
#         "financial_exchange where Xf_company_name = '%s' and exchange_good = '柴油 -35#' and year = '2018'" 

workbook = xlwt.Workbook(encoding='ascii')
worksheep_up = workbook.add_sheet('up', cell_overwrite_ok=True)
worksheep_up.write(0, 0, u'种子公司')
worksheep_up.write(0, 1, u'上游公司')
worksheep_up.write(0, 2, u'交易时间')
worksheep_up.write(0, 3, u'物品')
worksheep_up.write(0, 4, u'数量')
worksheep_up.write(0, 5, u'单价')
worksheep_up.write(0, 6, u'交易额')
worksheep_down = workbook.add_sheet('down', cell_overwrite_ok=True)
worksheep_down.write(0, 0, u'种子公司')
worksheep_down.write(0, 1, u'下游公司')
worksheep_down.write(0, 2, u'交易时间')
worksheep_down.write(0, 3, u'物品')
worksheep_down.write(0, 4, u'数量')
worksheep_down.write(0, 5, u'单价')
worksheep_down.write(0, 6, u'交易额')

up_num = 1
down_num = 1

for j in range(len(com_info)):
    print(j,com_info[j])
    cursor.execute(sql_up % com_info[j])
    data_up = cursor.fetchall()
    cursor.execute(sql_down % com_info[j])
    data_down = cursor.fetchall()
    if data_up:
        for v in data_up:
            com_name = v[0]
            date = str(v[1])
            good = v[2]
            Sl = v[3]
            Dj = v[4]
            Je = v[5]
            worksheep_up.write(up_num, 0, com_info[j])
            worksheep_up.write(up_num, 1, com_name)
            worksheep_up.write(up_num, 2, date)
            worksheep_up.write(up_num, 3, good)
            worksheep_up.write(up_num, 4, Sl)
            worksheep_up.write(up_num, 5, Dj)
            worksheep_up.write(up_num, 6, Je)
            up_num += 1
    if data_down:
        for v in data_down:
            com_name = v[0]
            date = str(v[1])
            good = v[2]
            Sl = v[3]
            Dj = v[4]
            Je = v[5]
            worksheep_down.write(down_num, 0, com_info[j])
            worksheep_down.write(down_num, 1, com_name)
            worksheep_down.write(down_num, 2, date)
            worksheep_down.write(down_num, 3, good)
            worksheep_down.write(down_num, 4, Sl)
            worksheep_down.write(down_num, 5, Dj)
            worksheep_down.write(down_num, 6, Je)
            down_num += 1

workbook.save('./files/20180917.xls')

In [7]:
#只给出单个厂家名称时

import pandas
import pymysql

sql_up = "select Xf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
         "financial_exchange where Gf_company_name = '%s' and year = '2018'"
sql_down = "select Gf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
         "financial_exchange where Xf_company_name = '%s' and year = '2018'"

connection = pymysql.connect(host = '47.92.25.70',user = 'root',password = 'Wfn031641',db = 'cxd_data',charset = 'utf8')

#c_name = '大连恒科石化有限公司'
c_name = '中油泰鸿（大连）石化有限公司'

writer=pandas.ExcelWriter('E:/开票信息/%s.xls' %c_name)
df_up = pandas.read_sql(sql_up %c_name,connection)
df_down = pandas.read_sql(sql_down %c_name,connection)
df_up.to_excel(writer, sheet_name='上游')
df_down.to_excel(writer, sheet_name='下游')
writer.save()
print('Done!')

Done!


In [8]:
#筛选上/下游特定企业

dir_station ='E:/开票信息/%s.xls' %c_name
#dir_station = r'D:\data\qichacha\oil_station\筛选后企业.xls'
df_s = pandas.read_excel(dir_station, sheet_name=0)

def select_station_name(df): 

    df_province = df
    select_list = []
    #del_list = []
    for i in range(len(df_province)):
        name = df_province.loc[i, 'Xf_company_name']
        if '华锦' in name or '蓝海' in name:
            select_list.append(i)
    select_unique = select_list        
    df_drop = df_province.drop(df_province.index[select_unique],axis=0,inplace=False) 
    df_need = df_province.loc[select_unique]
    return df_need, df_drop

df_need, df_drop = select_station_name(df_s)
station_dir =  r'E:/开票信息/%s筛选后企业.xls' %c_name
writer = pandas.ExcelWriter(station_dir)
df_need.to_excel(writer, sheet_name= 'need', index=False)
df_drop.to_excel(writer, sheet_name= 'drop', index=False)
writer.save()
print('筛选完成')

筛选完成


In [1]:
#导出符合条件厂家名称时

import pandas
import pymysql

sql_up = "select Xf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
         "financial_exchange where exchange_good = '柴油 -35#' and year = '2018'"
sql_down = "select Gf_company_name,exchange_date,exchange_good,Sl,Dj,Je from " \
         "financial_exchange where exchange_good = '柴油 -35#' and year = '2018'"

connection = pymysql.connect(host = '47.92.25.70',user = 'root',password = 'Wfn031641',db = 'cxd_data',charset = 'utf8')

writer=pandas.ExcelWriter('E:/20190307.xls')
df_up = pandas.read_sql(sql_up,connection)
df_down = pandas.read_sql(sql_down,connection)
df_up.to_excel(writer, sheet_name='上游')
df_down.to_excel(writer, sheet_name='下游')
writer.save()

In [15]:
#发票统计
import pandas
import pymysql
import xlwt
import xlrd

sql_cy = "select `Fphm`,`Gfmc`,`Spmc`,`Sl`,`Dj`,`Je`,`Kprq`,`Xfmc` from `ticket_bill` where `Spmc` like '%柴油%' and `Kprq` like '2018%'"
sql_qy = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%汽油%' and Kprq like '2018%'"
sql_jc = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%甲醇%' and Kprq like '2018%'"
sql_hm = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%航空煤%' and Kprq like '2018%'"
sql_sny = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%石脑油%' and Kprq like '2018%'"
sql_lq = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%沥青%' and Kprq like '2018%'"
sql_mtbe = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%MTBE%' and Kprq like '2018%'"
sql_zdw = "select Fphm,Gfmc,Spmc,Sl,Dj,Je,Kprq,Xfmc from ticket_bill where Spmc like '%正丁烷%' and Kprq like '2018%'"

connection = pymysql.connect(host = '47.92.25.70',user = 'root',password = 'Wfn031641',db = 'cxd_data',charset = 'utf8')
writer=pandas.ExcelWriter('E:/2018total.xls')

def write_title(sheet_name):
    filename=xlwt.Workbook()  
    sheet=filename.add_sheet(sheet_name)
    biaotou=['发票号码','上游企业','产品名称','数量','单价','金额','开票日期','下游企业']
    for i in range(0,len(biaotou)):  
        sheet.write(0,i,biaotou[i]) 
        
def write_down(sheet_name, sql):
    #writer=pandas.ExcelWriter('E:/2018total.xls')
    write_title(sheet_name)
    df = pandas.read_sql(sql,connection)
    df.to_excel(writer, sheet_name=sheet_name)    
    #writer.save()

#write_down('柴油', sql_cy)
#write_down('汽油', sql_qy)
write_down('甲醇', sql_jc)
write_down('航煤', sql_hm)
write_down('石脑油', sql_sny)
write_down('沥青', sql_lq)
write_down('MTBT', sql_mtbe)
write_down('正丁烷', sql_zdw)
writer.save()

In [16]:
df = pandas.read_sql(sql_cy,connection)
df.to_csv('E:/2018_chaiyou_total.csv', encoding = 'gb18030', mode = 'w', sep =',', index=False)

In [17]:
df = pandas.read_sql(sql_qy,connection)
df.to_csv('E:/2018_qiyou_total.csv', encoding = 'gb18030', mode = 'w', sep =',', index=False)